In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!pip install wikipedia
import wikipedia as wp
from bs4 import BeautifulSoup
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

  Stored in directory: /home/dsxuser/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-20

## Obtain data for Income and density

In [44]:
import urllib.request
url = "https://es.wikipedia.org/wiki/Anexo:Entidades_federativas_de_M%C3%A9xico_por_superficie,_poblaci%C3%B3n_y_densidad"
page = urllib.request.urlopen(url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")
all_tables=soup.find_all("table")
all_tables
right_table=soup.find('table', class_='sortable')
right_table
A=[]
B=[]
C=[]
D=[]
E=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==12:
        A.append(cells[4].find(text=True))
        B.append(cells[5].find(text=True))
        C.append(cells[6].find(text=True))
        D.append(cells[7].find(text=True))
        E.append(cells[8].find(text=True))
df=pd.DataFrame(A,columns=['State'])
df['Area']=B
df['%']=C
df['Pop']=D
df['Density']=E
df_density=df.drop(df.index[0])
df_density

,State,Area,%,Pop,Density
1,Chihuahua,247 455,"12,6",3 556 574,14
2,Sonora,179 355,"9,2",2 850 330,16
3,Coahuila de Zaragoza,151 562,"7,7",2 954 915,19
4,Durango,123 317,"6,3",1 754 754,14
5,Oaxaca,93 757,"4,8",3 967 889,42
6,Tamaulipas,80 249,"4,1",3 441 698,43
7,Jalisco,78 588,"4,0",7 844 830,100
8,Zacatecas,75 284,"3,8",1 579 209,21
9,Baja California Sur,73 909,"3,8",712 029,10
10,Chiapas,73 311,"3,7",5 217 908,71


In [91]:
import re
url = "https://es.wikipedia.org/wiki/Anexo:Estados_de_M%C3%A9xico_por_PIB_per_c%C3%A1pita"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
all_tables=soup.find_all("table")
all_tables
right_table=soup.find('table', class_='sortable')
right_table
A=[]
B=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==8:
        mlnk=cells[1].findAll('a')
        A.append(mlnk[1].contents[0])
        B.append(cells[7].find(text=True))
df=pd.DataFrame(A,columns=['State'])
df['Income']=B

for index in df.index:
    df.loc[index,'Income']=df.loc[index,'Income'].replace(' ','')
    df.loc[index,'Income']=df.loc[index,'Income'].replace('\n','')
    df.loc[index,'Income']=df.loc[index,'Income'].replace('\xa0','')
df_income=df
df_income

,State,Income
0,Nacional,114919
1,Campeche,628311
2,Ciudad de México,257121
3,Nuevo León,205952
4,Tabasco,179044
5,Coahuila,159651
6,Querétaro,156244
7,Quintana Roo,149983
8,Sonora,139712
9,Baja California Sur,139599


## Dataframe with information of each city

In [121]:
column_names = ["city","lat", "lon", "density","income","mean"]
df_cities = pd.DataFrame(columns = column_names)

In [122]:
float(df_income['Income'][df_income['State'] == 'Jalisco'])

112825.0

In [123]:
address = 'Mexico City'
geolocator = Nominatim(user_agent="cdmx")
loc = geolocator.geocode(address)
df_cities.loc[0,"lat"] = loc.latitude
df_cities.loc[0,"lon"] = loc.longitude
df_cities.loc[0,"density"]=float(df_density['Density'][df_density['State'] == 'Ciudad de México'])
df_cities.loc[0,"income"]=float(df_income['Income'][df_income['State'] == 'Ciudad de México'])
df_cities.loc[0,"city"]=address
address = 'Monterrey'
geolocator = Nominatim(user_agent="cdmx")
loc = geolocator.geocode(address)
df_cities.loc[1,"lat"] = loc.latitude
df_cities.loc[1,"lon"] = loc.longitude
df_cities.loc[1,"density"]=float(df_density['Density'][df_density['State'] == 'Nuevo León'])
df_cities.loc[1,"income"]=float(df_income['Income'][df_income['State'] == 'Nuevo León'])
df_cities.loc[1,"city"]=address
address = 'Guadalajara'
geolocator = Nominatim(user_agent="cdmx")
loc = geolocator.geocode(address)
df_cities.loc[2,"lat"] = loc.latitude
df_cities.loc[2,"lon"] = loc.longitude
df_cities.loc[2,"density"]=float(df_density['Density'][df_density['State'] == 'Jalisco'])
df_cities.loc[2,"income"]=float(df_income['Income'][df_income['State'] == 'Jalisco'])
df_cities.loc[2,"city"]=address

df_cities

,city,lat,lon,density,income,mean
0,Mexico City,19.4326,-99.1332,5966,257121,NaN
1,Monterrey,25.6802,-100.315,80,205952,NaN
2,Guadalajara,20.672,-103.338,100,112825,NaN


## Obtain coffee shop data for each city

In [95]:
CLIENT_ID = '2GGFMVZVQKU0A3LZOSD4VDZY0I0ZQXPL3ISUPHYUV1XGF40D' # your Foursquare ID
CLIENT_SECRET = '02T1XNVL2MGZJVX00PJRAYKQD0BRFDNHFTIUI1WQJNTA0VWX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2GGFMVZVQKU0A3LZOSD4VDZY0I0ZQXPL3ISUPHYUV1XGF40D
CLIENT_SECRET:02T1XNVL2MGZJVX00PJRAYKQD0BRFDNHFTIUI1WQJNTA0VWX


In [105]:
# type your answer here
LIMIT = 100
cities = ["Mexico City", 'Monterrey', 'Guadalajara']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1e0931735") # COFFEE SHOP CATEGORY ID
    results[city] = requests.get(url).json()
results

{'Mexico City': {'meta': {'code': 200,
   'requestId': '5ea5179683525f001b3c2076'},
  'response': {'suggestedFilters': {'header': 'Tap to show:',
    'filters': [{'name': 'Open now', 'key': 'openNow'},
     {'name': 'With specials', 'key': 'specials'}]},
   'geocode': {'what': '',
    'where': 'mexico city',
    'center': {'lat': 19.42847, 'lng': -99.12766},
    'displayString': 'Mexico City, DF, Mexico',
    'cc': 'MX',
    'geometry': {'bounds': {'ne': {'lat': 19.515304989460464,
       'lng': -99.05579900650167},
      'sw': {'lat': 19.356858007471764, 'lng': -99.25983899084375}}},
    'slug': 'mexico-city',
    'longId': '72057594041458533'},
   'headerLocation': 'Mexico City',
   'headerFullLocation': 'Mexico City',
   'headerLocationGranularity': 'city',
   'query': 'coffee shop',
   'totalResults': 229,
   'suggestedBounds': {'ne': {'lat': 19.490494882479943,
     'lng': -99.06418412387383},
    'sw': {'lat': 19.3476273540534, 'lng': -99.21828280257141}},
   'groups': [{'type': 

In [106]:
df_venues={}
for city in df_cities['city']:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

## Create Map for each city with coffee shops

In [112]:
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    print(f"Total number of coffee shops in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of coffee shops in Mexico City =  229
Showing Top 100
Total number of coffee shops in Monterrey =  121
Showing Top 100
Total number of coffee shops in Guadalajara =  171
Showing Top 100


In [124]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])
    df_cities['mean'][df_cities['city'] == city]=np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values))
    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Mexico City
Mean Distance from Mean coordinates
0.027396339979634133
Monterrey
Mean Distance from Mean coordinates
0.04303317086578117
Guadalajara
Mean Distance from Mean coordinates
0.01917203828109947


In [116]:
maps[cities[0]]

In [117]:
maps[cities[1]]

In [144]:
maps[cities[2]]

# Normalize data and obtain results

Get inverse of income

In [139]:
df=df_cities
df['income']=1/df_cities['income']
df

,city,lat,lon,density,income,mean
0,Mexico City,19.4326,-99.1332,5966,3.88922e-06,0.0273963
1,Monterrey,25.6802,-100.315,80,4.8555e-06,0.0430332
2,Guadalajara,20.672,-103.338,100,8.86328e-06,0.019172


In [145]:
from sklearn import preprocessing

x = df[['density','income','mean']].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_norm = pd.DataFrame(x_scaled).transpose()
df_norm=df_norm.mean()
column_names = ["city","indicator"]
df_final = pd.DataFrame(columns = column_names)
df_final['city']=df_cities['city']
df_final['indicator']=df_norm
df_final

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


,city,indicator
0,Mexico City,0.448225
1,Monterrey,0.398088
2,Guadalajara,0.334466


# Which tells us that Guadalajara is the best option to open a Coffee Shop